In [1]:
from keras_efficientmixnets import *
# https://github.com/tensorflow/tpu/blob/master/models/official/mnasnet/mixnet/mixnet_model.py
# https://arxiv.org/abs/1908.04008


Using TensorFlow backend.


In [2]:
def f2(p, r):
    return 5 * ((p * r) / ((4 * p) + r))

In [4]:
f2(91.8, 99.99)

98.23714120593334

In [2]:
from keras.models import Sequential, Model
from keras.layers import *
import numpy as np
import json


In [29]:
# If a GPU is busy, use the available one
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID";
 
# The GPU id to use, usually either "0" or "1";
os.environ["CUDA_VISIBLE_DEVICES"]="1";  

!export CUDA_VISIBLE_DEVICES=1

In [11]:
import json
with open('Notebooks/partition_fs.json', 'r') as f:
    partition = json.load(f)

In [3]:
with open('Data/partition_chr.json', 'r') as f:
    partition = json.load(f)

In [4]:
len(partition["paths_labels_chr22_down"])

27344

In [5]:
list_indx_train_downsampled = partition['train_downsampled']
list_indx_valid_downsampled = partition['valid_downsampled']
list_indx_test_downsampled  = partition['test_downsampled']
list_indx_test  = partition['test']

In [31]:
frombuffer = np.frombuffer
empty = np.empty
zeros = np.zeros

def read_pgm_ref(filename):
    
    """Return germline and somatic images (with 5 ref rows in each channel) data 
       from a raw PGM file as two numpy arrays of shape (152, 299, 5)
    
    Format specification: http://netpbm.sourceforge.net/doc/pgm.html

    """
    with open(filename, 'rb') as f:
        X = frombuffer(f.read(), dtype='u1',
                                count=447005,
                                offset=63).reshape(-1, 299)

        tensor = empty((299, 299, 5), dtype='float32')

        tensor[:, :, 0] = X[:299, ]     # bases
        tensor[:, :, 1] = X[299:598, ]  # insertions
        tensor[:, :, 2] = X[598:897, ]  # deletions
        tensor[:, :, 3] = X[897:1196, ] # mismatchs
        tensor[:, :, 4] = X[1196:, ]    # base quality

        return tensor
    
    

def load_data_set_from_pileups(list_indx):
    '''
     load in memory the dataset from a precomputed list of index 
     
     '''
    N = len(list_indx)
    print("Size of dataset : ", N)
    X = zeros((N, 299, 299, 5), dtype='float32')
    y = zeros((N, 2), dtype='float32')
    
    for i, item in enumerate(list_indx):
        X[i] = read_pgm_ref(item[0])
        y[i] = item[1]
        if i%1000 == 0 and i>0:
            print("Loaded {} images in memory, {} left".format(i, N-i))
    return X, y



    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples' # X : (n_samples, 299, 299, 5)
        # Initialization
        X = empty((self.batch_size, 299, 299, 5))
        y = empty((self.batch_size, 2))

        # Generate data
        for i, item in enumerate(list_IDs_temp):
            X[i, ] = read_pgm_ref(item[0])
            # Store class
            y[i] = item[1]
        return X, y

In [32]:
a = {"a" : 55}

for key, item in a.items():
    print(key, item)

a 55


In [33]:
list(a.items())[0]

('a', 55)

In [43]:
floor = np.floor
shuffle = np.random.shuffle

from keras.utils import Sequence
# va être utilisée pour tous les chromosomes
class DataGenerator(Sequence):
    'Generates data for Keras'
    def __init__(self, dic_paths_labels, batch_size=32, shuffle=True):
        'Initialization'
        self.batch_size = batch_size
        self.list_IDs = list(dic_paths_labels.items()) # [(path, label), ...]
        self.shuffle = shuffle
        self.datagen = self.__data_generation
        self.on_epoch_end()
  
    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index * self.batch_size : (index + 1) * self.batch_size]
        # Find list of IDs
        list_IDs_temp = [self.list_IDs[k] for k in indexes]
        # Generate data
        return self.datagen(list_IDs_temp)

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples' # X : (n_samples, 299, 299, 5)
        # Initialization
        X = empty((self.batch_size, 299, 299, 5))
        y = empty((self.batch_size, 2))

        # Generate data
        for i, item in enumerate(list_IDs_temp):
            X[i, ] = read_pgm_ref(item[0])
            # Store class
            y[i] = item[1]
        return X, y

In [44]:
data = DataGenerator(partition["paths_labels_chr1_down"])

In [49]:
chromosom = ["chr1", "chr21"]
d = True

keys = [partition["paths_labels_" + chrom + "_down"] if d else partition["paths_labels_" + chrom] for chrom in chromosom]

In [51]:
a = {}

for d in keys:
    a.update(d)

In [57]:
import numpy as np
from sklearn.model_selection import train_test_split

locs_train, locs_validation = train_test_split(list(a.keys()), test_size=0.2, random_state=1997)


In [59]:
locs_validation

['/mnt/fast_storage/calva/pileups/mixture_HG001_HG002/chr1/15/chr1_15441040.pgm',
 '/mnt/fast_storage/calva/pileups/mixture_HG001_HG002/chr1/6/chr1_6945489.pgm',
 '/mnt/fast_storage/calva/pileups/mixture_HG001_HG002/chr1/5/chr1_5052588.pgm',
 '/mnt/fast_storage/calva/pileups/mixture_HG001_HG002/chr1/181/chr1_181211100.pgm',
 '/mnt/fast_storage/calva/pileups/mixture_HG001_HG002/chr1/185/chr1_185698544.pgm',
 '/mnt/fast_storage/calva/pileups/mixture_HG001_HG002/chr21/42/chr21_42718938.pgm',
 '/mnt/fast_storage/calva/pileups/mixture_HG001_HG002/chr1/170/chr1_170926657.pgm',
 '/mnt/fast_storage/calva/pileups/mixture_HG001_HG002/chr1/41/chr1_41797073.pgm',
 '/mnt/fast_storage/calva/pileups/mixture_HG001_HG002/chr1/241/chr1_241681882.pgm',
 '/mnt/fast_storage/calva/pileups/mixture_HG001_HG002/chr1/15/chr1_15601898.pgm',
 '/mnt/fast_storage/calva/pileups/mixture_HG001_HG002/chr1/24/chr1_24874064.pgm',
 '/mnt/fast_storage/calva/pileups/mixture_HG001_HG002/chr21/16/chr21_16399658.pgm',
 '/mnt/f

In [53]:
len(a)

210414

In [45]:
for i in data:
    print(i)

(array([[[[127.,   0.,   0.,   0., 255.],
         [127.,   0.,   0.,   0., 255.],
         [127.,   0.,   0.,   0., 255.],
         ...,
         [ 63.,   0.,   0.,   0., 255.],
         [ 63.,   0.,   0.,   0., 255.],
         [127.,   0.,   0.,   0., 255.]],

        [[127.,   0.,   0.,   0., 255.],
         [127.,   0.,   0.,   0., 255.],
         [127.,   0.,   0.,   0., 255.],
         ...,
         [ 63.,   0.,   0.,   0., 255.],
         [ 63.,   0.,   0.,   0., 255.],
         [127.,   0.,   0.,   0., 255.]],

        [[127.,   0.,   0.,   0., 255.],
         [127.,   0.,   0.,   0., 255.],
         [127.,   0.,   0.,   0., 255.],
         ...,
         [ 63.,   0.,   0.,   0., 255.],
         [ 63.,   0.,   0.,   0., 255.],
         [127.,   0.,   0.,   0., 255.]],

        ...,

        [[  0.,   0.,   0.,   0.,   0.],
         [  0.,   0.,   0.,   0.,   0.],
         [  0.,   0.,   0.,   0.,   0.],
         ...,
         [  0.,   0.,   0.,   0.,   0.],
         [  0.,   0.,

(array([[[[127.,   0.,   0.,   0., 255.],
         [127.,   0.,   0.,   0., 255.],
         [191.,   0.,   0.,   0., 255.],
         ...,
         [  0.,   0.,   0.,   0.,   0.],
         [  0.,   0.,   0.,   0.,   0.],
         [  0.,   0.,   0.,   0.,   0.]],

        [[127.,   0.,   0.,   0., 255.],
         [127.,   0.,   0.,   0., 255.],
         [191.,   0.,   0.,   0., 255.],
         ...,
         [  0.,   0.,   0.,   0.,   0.],
         [  0.,   0.,   0.,   0.,   0.],
         [  0.,   0.,   0.,   0.,   0.]],

        [[127.,   0.,   0.,   0., 255.],
         [127.,   0.,   0.,   0., 255.],
         [191.,   0.,   0.,   0., 255.],
         ...,
         [  0.,   0.,   0.,   0.,   0.],
         [  0.,   0.,   0.,   0.,   0.],
         [  0.,   0.,   0.,   0.,   0.]],

        ...,

        [[  0.,   0.,   0.,   0.,   0.],
         [  0.,   0.,   0.,   0.,   0.],
         [  0.,   0.,   0.,   0.,   0.],
         ...,
         [  0.,   0.,   0.,   0.,   0.],
         [  0.,   0.,

(array([[[[127.,   0.,   0.,   0., 255.],
         [255.,   0.,   0.,   0., 255.],
         [127.,   0.,   0.,   0., 255.],
         ...,
         [  0.,   0.,   0.,   0.,   0.],
         [  0.,   0.,   0.,   0.,   0.],
         [  0.,   0.,   0.,   0.,   0.]],

        [[127.,   0.,   0.,   0., 255.],
         [255.,   0.,   0.,   0., 255.],
         [127.,   0.,   0.,   0., 255.],
         ...,
         [  0.,   0.,   0.,   0.,   0.],
         [  0.,   0.,   0.,   0.,   0.],
         [  0.,   0.,   0.,   0.,   0.]],

        [[127.,   0.,   0.,   0., 255.],
         [255.,   0.,   0.,   0., 255.],
         [127.,   0.,   0.,   0., 255.],
         ...,
         [  0.,   0.,   0.,   0.,   0.],
         [  0.,   0.,   0.,   0.,   0.],
         [  0.,   0.,   0.,   0.,   0.]],

        ...,

        [[  0.,   0.,   0.,   0.,   0.],
         [  0.,   0.,   0.,   0.,   0.],
         [  0.,   0.,   0.,   0.,   0.],
         ...,
         [  0.,   0.,   0.,   0.,   0.],
         [  0.,   0.,

(array([[[[191.,   0.,   0.,   0., 255.],
         [127.,   0.,   0.,   0., 255.],
         [127.,   0.,   0.,   0., 255.],
         ...,
         [ 63.,   0.,   0.,   0., 255.],
         [255.,   0.,   0.,   0., 255.],
         [191.,   0.,   0.,   0., 255.]],

        [[191.,   0.,   0.,   0., 255.],
         [127.,   0.,   0.,   0., 255.],
         [127.,   0.,   0.,   0., 255.],
         ...,
         [ 63.,   0.,   0.,   0., 255.],
         [255.,   0.,   0.,   0., 255.],
         [191.,   0.,   0.,   0., 255.]],

        [[191.,   0.,   0.,   0., 255.],
         [127.,   0.,   0.,   0., 255.],
         [127.,   0.,   0.,   0., 255.],
         ...,
         [ 63.,   0.,   0.,   0., 255.],
         [255.,   0.,   0.,   0., 255.],
         [191.,   0.,   0.,   0., 255.]],

        ...,

        [[  0.,   0.,   0.,   0.,   0.],
         [  0.,   0.,   0.,   0.,   0.],
         [  0.,   0.,   0.,   0.,   0.],
         ...,
         [  0.,   0.,   0.,   0.,   0.],
         [  0.,   0.,

(array([[[[ 63.,   0.,   0.,   0., 255.],
         [191.,   0.,   0.,   0., 255.],
         [ 63.,   0.,   0.,   0., 255.],
         ...,
         [255.,   0.,   0.,   0., 255.],
         [ 63.,   0.,   0.,   0., 255.],
         [255.,   0.,   0.,   0., 255.]],

        [[ 63.,   0.,   0.,   0., 255.],
         [191.,   0.,   0.,   0., 255.],
         [ 63.,   0.,   0.,   0., 255.],
         ...,
         [255.,   0.,   0.,   0., 255.],
         [ 63.,   0.,   0.,   0., 255.],
         [255.,   0.,   0.,   0., 255.]],

        [[ 63.,   0.,   0.,   0., 255.],
         [191.,   0.,   0.,   0., 255.],
         [ 63.,   0.,   0.,   0., 255.],
         ...,
         [255.,   0.,   0.,   0., 255.],
         [ 63.,   0.,   0.,   0., 255.],
         [255.,   0.,   0.,   0., 255.]],

        ...,

        [[  0.,   0.,   0.,   0.,   0.],
         [  0.,   0.,   0.,   0.,   0.],
         [  0.,   0.,   0.,   0.,   0.],
         ...,
         [  0.,   0.,   0.,   0.,   0.],
         [  0.,   0.,

...

(array([[[[127.,   0.,   0.,   0., 255.],
         [255.,   0.,   0.,   0., 255.],
         [191.,   0.,   0.,   0., 255.],
         ...,
         [255.,   0.,   0.,   0., 255.],
         [255.,   0.,   0.,   0., 255.],
         [255.,   0.,   0.,   0., 255.]],

        [[127.,   0.,   0.,   0., 255.],
         [255.,   0.,   0.,   0., 255.],
         [191.,   0.,   0.,   0., 255.],
         ...,
         [255.,   0.,   0.,   0., 255.],
         [255.,   0.,   0.,   0., 255.],
         [255.,   0.,   0.,   0., 255.]],

        [[127.,   0.,   0.,   0., 255.],
         [255.,   0.,   0.,   0., 255.],
         [191.,   0.,   0.,   0., 255.],
         ...,
         [255.,   0.,   0.,   0., 255.],
         [255.,   0.,   0.,   0., 255.],
         [255.,   0.,   0.,   0., 255.]],

        ...,

        [[  0.,   0.,   0.,   0.,   0.],
         [  0.,   0.,   0.,   0.,   0.],
         [  0.,   0.,   0.,   0.,   0.],
         ...,
         [  0.,   0.,   0.,   0.,   0.],
         [  0.,   0.,

(array([[[[ 63.,   0.,   0.,   0., 255.],
         [ 63.,   0.,   0.,   0., 255.],
         [255.,   0.,   0.,   0., 255.],
         ...,
         [191.,   0.,   0.,   0., 255.],
         [ 63.,   0.,   0.,   0., 255.],
         [  0.,   0.,   0.,   0.,   0.]],

        [[ 63.,   0.,   0.,   0., 255.],
         [ 63.,   0.,   0.,   0., 255.],
         [255.,   0.,   0.,   0., 255.],
         ...,
         [191.,   0.,   0.,   0., 255.],
         [ 63.,   0.,   0.,   0., 255.],
         [  0.,   0.,   0.,   0.,   0.]],

        [[ 63.,   0.,   0.,   0., 255.],
         [ 63.,   0.,   0.,   0., 255.],
         [255.,   0.,   0.,   0., 255.],
         ...,
         [191.,   0.,   0.,   0., 255.],
         [ 63.,   0.,   0.,   0., 255.],
         [  0.,   0.,   0.,   0.,   0.]],

        ...,

        [[  0.,   0.,   0.,   0.,   0.],
         [  0.,   0.,   0.,   0.,   0.],
         [  0.,   0.,   0.,   0.,   0.],
         ...,
         [  0.,   0.,   0.,   0.,   0.],
         [  0.,   0.,

(array([[[[191.,   0.,   0.,   0., 255.],
         [255.,   0.,   0.,   0., 255.],
         [ 63.,   0.,   0.,   0., 255.],
         ...,
         [ 63.,   0.,   0.,   0., 255.],
         [ 63.,   0.,   0.,   0., 255.],
         [191.,   0.,   0.,   0., 255.]],

        [[191.,   0.,   0.,   0., 255.],
         [255.,   0.,   0.,   0., 255.],
         [ 63.,   0.,   0.,   0., 255.],
         ...,
         [ 63.,   0.,   0.,   0., 255.],
         [ 63.,   0.,   0.,   0., 255.],
         [191.,   0.,   0.,   0., 255.]],

        [[191.,   0.,   0.,   0., 255.],
         [255.,   0.,   0.,   0., 255.],
         [ 63.,   0.,   0.,   0., 255.],
         ...,
         [ 63.,   0.,   0.,   0., 255.],
         [ 63.,   0.,   0.,   0., 255.],
         [191.,   0.,   0.,   0., 255.]],

        ...,

        [[  0.,   0.,   0.,   0.,   0.],
         [  0.,   0.,   0.,   0.,   0.],
         [  0.,   0.,   0.,   0.,   0.],
         ...,
         [ 63.,   0.,   0.,   0., 115.],
         [ 63.,   0.,

(array([[[[255.,   0.,   0.,   0., 255.],
         [191.,   0.,   0.,   0., 255.],
         [127.,   0.,   0.,   0., 255.],
         ...,
         [ 63.,   0.,   0.,   0., 255.],
         [ 63.,   0.,   0.,   0., 255.],
         [ 63.,   0.,   0.,   0., 255.]],

        [[255.,   0.,   0.,   0., 255.],
         [191.,   0.,   0.,   0., 255.],
         [127.,   0.,   0.,   0., 255.],
         ...,
         [ 63.,   0.,   0.,   0., 255.],
         [ 63.,   0.,   0.,   0., 255.],
         [ 63.,   0.,   0.,   0., 255.]],

        [[255.,   0.,   0.,   0., 255.],
         [191.,   0.,   0.,   0., 255.],
         [127.,   0.,   0.,   0., 255.],
         ...,
         [ 63.,   0.,   0.,   0., 255.],
         [ 63.,   0.,   0.,   0., 255.],
         [ 63.,   0.,   0.,   0., 255.]],

        ...,

        [[  0.,   0.,   0.,   0.,   0.],
         [  0.,   0.,   0.,   0.,   0.],
         [  0.,   0.,   0.,   0.,   0.],
         ...,
         [  0.,   0.,   0.,   0.,   0.],
         [  0.,   0.,

KeyboardInterrupt: 

In [7]:
path_train = "/mnt/fast_storage/calva/pileups/mixture_HG001_HG002/chr21/"
path_test = "/mnt/fast_storage/calva/pileups/mixture_HG001_HG002/chr22/"
labels_chr21_encoded = partition['labels_chr21_encoded']
labels_chr22_encoded = partition['labels_chr22_encoded']

In [8]:
labels_chr21_encoded = {int(i) : j for i, j in labels_chr21_encoded.items()}
labels_chr22_encoded = {int(i) : j for i, j in labels_chr22_encoded.items()}

In [9]:
# X_train, y_train = load_data_set_from_pileups(path_train, list_indx_train_downsampled, labels_chr21_encoded)
# X_valid, y_valid = load_data_set_from_pileups(path_train, list_indx_valid_downsampled, labels_chr21_encoded)
X_test, y_test   = load_data_set_from_pileups(path_test,  list_indx_test,  labels_chr22_encoded, test=True)

Size of dataset :  59357
Loaded 1000 images in memory, 58357 left
Loaded 2000 images in memory, 57357 left
Loaded 3000 images in memory, 56357 left
Loaded 4000 images in memory, 55357 left
Loaded 5000 images in memory, 54357 left
Loaded 6000 images in memory, 53357 left
Loaded 7000 images in memory, 52357 left
Loaded 8000 images in memory, 51357 left
Loaded 9000 images in memory, 50357 left
Loaded 10000 images in memory, 49357 left
Loaded 11000 images in memory, 48357 left
Loaded 12000 images in memory, 47357 left
Loaded 13000 images in memory, 46357 left
Loaded 14000 images in memory, 45357 left
Loaded 15000 images in memory, 44357 left
Loaded 16000 images in memory, 43357 left
Loaded 17000 images in memory, 42357 left
Loaded 18000 images in memory, 41357 left
Loaded 19000 images in memory, 40357 left
Loaded 20000 images in memory, 39357 left
Loaded 21000 images in memory, 38357 left
Loaded 22000 images in memory, 37357 left
Loaded 23000 images in memory, 36357 left
Loaded 24000 image

In [1]:
path_partition = 'Data/partition_chr.json'

In [2]:
!python training.py -p {path_partition} --downsample --no_memory --architecture "EfficientNetB2" --chromosom "chr1" "chr2" "chr21" --mixed -opt "RAdam" --lookahead -b 64 -e 100

Using TensorFlow backend.
/opt/python3.5/lib/python3.5/site-packages/keras_applications/imagenet_utils.py:279: UserWarning: This model usually expects 1 or 3 input channels. However, it was passed an input_shape with 5 input channels.
  str(input_shape[-1]) + ' input channels.')
2019-08-19 17:33:32.914287: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1432] Found device 0 with properties: 
name: Quadro M6000 24GB major: 5 minor: 2 memoryClockRate(GHz): 1.114
pciBusID: 0000:03:00.0
totalMemory: 23.90GiB freeMemory: 23.63GiB
2019-08-19 17:33:32.914355: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1511] Adding visible gpu devices: 0
2019-08-19 17:33:33.356962: I tensorflow/core/common_runtime/gpu/gpu_device.cc:982] Device interconnect StreamExecutor with strength 1 edge matrix:
2019-08-19 17:33:33.357048: I tensorflow/core/common_runtime/gpu/gpu_device.cc:988]      0 
2019-08-19 17:33:33.357060: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1001] 0:   N 
2019-08-19 17:33:3

In [11]:
def create_network(input_shape, keras_model, mixed=True):
    '''Conv Network architecture, based on a keras model such as EfficientNets
    '''
    inp = Input(shape=input_shape)
    base_model = keras_model(input_tensor=inp, weights=None, include_top=False, input_shape=input_shape, pooling="avg", mixed=mixed)
    x = base_model.output
    x = Dense(1024, use_bias=True)(x)
    x = Swish()(x)
    x = Dropout(0.5)(x)
    x = Dense(200, use_bias=True)(x)
    x = Swish()(x)
    x = Dropout(0.2)(x)
    predictions = Dense(2, activation="softmax")(x)
    return Model(inp, predictions)

In [13]:
from keras_efficientmixnets.custom_objects import Swish
model = create_network((299, 299, 5), EfficientNetB0, True)

/opt/python3.5/lib/python3.5/site-packages/keras_applications/imagenet_utils.py:279: UserWarning: This model usually expects 1 or 3 input channels. However, it was passed an input_shape with 5 input channels.
  str(input_shape[-1]) + ' input channels.')


In [14]:
model.load_weights("model_EfficientNetB0.h5")

In [15]:
y_pred = model.predict(X_test, verbose=1)

59357/59357 [==============================] - 615s 10ms/step


In [18]:
import numpy as np
y_pred2 = np.argmax(y_pred, 1)
y_test2 = np.argmax(y_test, 1)

np.sum(y_pred2 == y_test2) / len(y_pred2)

0.978873595363647

In [25]:
from sklearn.metrics import f1_score, precision_recall_curve

In [20]:
f1_score(y_test2, y_pred2)

0.9549050632911393

In [22]:
y_pred_prob = []
for i in y_pred:
    if np.argmax(i) == 0:
        y_pred_prob.append(1 - i[0])
    else:
        y_pred_prob.append(i[np.argmax(i)])

y_pred_prob = np.array(y_pred_prob)
y_test_bin  = np.argmax(y_test, axis=1)

In [27]:
precision, recall, tr = precision_recall_curve(y_test_bin, y_pred_prob)

In [129]:
!python eval_model.py -p {path_partition} --architecture "EfficientNetB2" --mixed --path_model "EfficientNetB2_mixed_adam_lookahead.h5" -chr "chr22"

Using TensorFlow backend.
Size of dataset :  27344
Loaded 1000 images in memory, 26344 left
Loaded 2000 images in memory, 25344 left
^C
Traceback (most recent call last):
  File "eval_model.py", line 157, in <module>
    eval_model(args)
  File "eval_model.py", line 65, in eval_model
    X_test, y_test = load_data_set_from_pileups(paths_labels)
  File "/home/users/cec1/utils.py", line 47, in load_data_set_from_pileups
    X[i] = read_pgm_ref(item[0])
KeyboardInterrupt


In [97]:
from collections import Counter

count_maj = 0
count_min = 0

for i, j in partition['paths_labels_chr22'].items():
    if j == [0., 1.]:
        count_maj += 1
    elif j == [1., 0.]:
        count_min += 1
        

In [98]:
count_maj

13672

In [99]:
count_min

45685

In [96]:
len(partition['paths_labels_chr22_down'])

27344

In [108]:
from datetime import date

In [110]:
str(date.today())

'2019-08-19'

In [3]:
import numpy as np 
# dummy training set
# X_train = np.array([np.random.rand(64, 64, 3) for _ in range(10)])
X_train = np.array([np.ones((64, 64, 3)) for _ in range(10)])
y_train = np.array([np.random.choice([0, 1]) for _ in range(10)])
t = np.array([np.random.rand(64,64,3)])

In [4]:
from keras import layers
from keras.initializers import Constant
from keras.engine.base_layer import InputSpec

class AttentiveBatchNormalization(layers.BatchNormalization):
    def __init__(self, momentum=0.99, epsilon=0.001, axis=-1, **kwargs):
        super(layers.BatchNormalization, self).__init__(**kwargs)
        self.momentum = momentum
        self.epsilon  = epsilon
        self.axis = axis
        
    def build(self, input_shape):
        if len(input_shape) != 4:
            raise ValueError('expected 4D input (got {}D input)'.format(input_shape))
            
        dim = input_shape[self.axis]
      
        shape = (dim, )
    
        self.weight = self.add_weight(name='weight', 
                                      shape=shape,
                                      initializer=Constant(1),
                                      trainable=True)

        self.bias = self.add_weight(name='bias', 
                                    shape=shape,
                                    initializer=Constant(0),
                                    trainable=True)

        self.weight_readjust = self.add_weight(name='weight_readjust', 
                                               shape=shape,
                                               initializer=Constant(0),
                                               trainable=True)
        
        self.bias_readjust = self.add_weight(name='bias_readjust', 
                                             shape=shape,
                                             initializer=Constant(-1),
                                             trainable=True)
        

        super(layers.BatchNormalization, self).build(input_shape)


    def call(self, input):
        if self.axis == -1:
            data_format = 'channels_last'
        else:
            data_format = 'channel_first'
            
        avg = layers.GlobalAveragePooling2D(data_format)(input) # (batch_size, n_channels)
        attention = K.sigmoid(avg * self.weight_readjust + self.bias_readjust)

        bn_weights = self.weight * attention
        
        out_bn = layers.BatchNormalization(momentum=self.momentum, epsilon=self.epsilon, 
                                           axis=self.axis, center=False, scale=False)(input)
        
        if K.int_shape(input)[0] is None or K.int_shape(input)[0] > 1:
            bn_weights = bn_weights[:, None, None, :]
            self.bias  = self.bias[None, None, None, :]
            
        out_bn = out_bn * bn_weights + self.bias
 
        return out_bn

    def get_config(self):
        config = {
            'axis' : self.axis,
            'momentum' : self.momentum,
            'epsilon' : self.epsilon
        }
        base_config = super(BatchNormalization, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

In [8]:
att = AttentiveBatchNormalization()

In [9]:
att(K.X_train)

ValueError: Layer attentive_batch_normalization_2 was called with an input that isn't a symbolic tensor. Received type: <class 'numpy.ndarray'>. Full input: [array([[[[1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.],
         ...,
         [1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.]],

        [[1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.],
         ...,
         [1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.]],

        [[1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.],
         ...,
         [1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.]],

        ...,

        [[1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.],
         ...,
         [1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.]],

        [[1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.],
         ...,
         [1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.]],

        [[1., 1., 1.],
         [1., 1., 1.],
         [1.

In [5]:
from keras.models import Model

inp = Input((64, 64, 3))
x = AttentiveBatchNormalization()(inp)
model = Model(inp, x)

In [7]:
model.predict(X_train)

array([[[[0.26880705, 0.26880705, 0.26880705],
         [0.26880705, 0.26880705, 0.26880705],
         [0.26880705, 0.26880705, 0.26880705],
         ...,
         [0.26880705, 0.26880705, 0.26880705],
         [0.26880705, 0.26880705, 0.26880705],
         [0.26880705, 0.26880705, 0.26880705]],

        [[0.26880705, 0.26880705, 0.26880705],
         [0.26880705, 0.26880705, 0.26880705],
         [0.26880705, 0.26880705, 0.26880705],
         ...,
         [0.26880705, 0.26880705, 0.26880705],
         [0.26880705, 0.26880705, 0.26880705],
         [0.26880705, 0.26880705, 0.26880705]],

        [[0.26880705, 0.26880705, 0.26880705],
         [0.26880705, 0.26880705, 0.26880705],
         [0.26880705, 0.26880705, 0.26880705],
         ...,
         [0.26880705, 0.26880705, 0.26880705],
         [0.26880705, 0.26880705, 0.26880705],
         [0.26880705, 0.26880705, 0.26880705]],

        ...,

        [[0.26880705, 0.26880705, 0.26880705],
         [0.26880705, 0.26880705, 0.26880705]

In [294]:
dim = K.int_shape(input)[-1]

shape = (dim, )

weight = K.constant(1, shape=shape)
print(K.int_shape(weight))

bias = K.constant(0, shape=shape)
print(K.int_shape(bias))


weight_readjust = K.constant(0, shape=shape)
print(K.int_shape(weight_readjust))

bias_readjust = K.constant(-1, shape=shape)
print(K.int_shape(bias_readjust))


(3,)
(3,)
(3,)
(3,)


In [295]:
avg = layers.GlobalAveragePooling2D()(input)
print(K.int_shape(avg))

(10, 3)


In [296]:
K.eval(avg)

array([[1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.]], dtype=float32)

In [297]:
attention = (avg * weight_readjust + bias_readjust)
print(K.int_shape(attention))

(10, 3)


In [298]:
K.eval(attention)

array([[-1., -1., -1.],
       [-1., -1., -1.],
       [-1., -1., -1.],
       [-1., -1., -1.],
       [-1., -1., -1.],
       [-1., -1., -1.],
       [-1., -1., -1.],
       [-1., -1., -1.],
       [-1., -1., -1.],
       [-1., -1., -1.]], dtype=float32)

In [300]:
bn_weights = weight * attention
print(K.int_shape(bn_weights))
K.eval(bn_weights)

(10, 3)


array([[-1., -1., -1.],
       [-1., -1., -1.],
       [-1., -1., -1.],
       [-1., -1., -1.],
       [-1., -1., -1.],
       [-1., -1., -1.],
       [-1., -1., -1.],
       [-1., -1., -1.],
       [-1., -1., -1.],
       [-1., -1., -1.]], dtype=float32)

In [301]:
out_bn = layers.BatchNormalization(center=False, scale=False)(input)
print(K.int_shape(out_bn))
print(K.eval(out_bn))

(10, 64, 64, 3)
[[[[0.99950033 0.99950033 0.99950033]
   [0.99950033 0.99950033 0.99950033]
   [0.99950033 0.99950033 0.99950033]
   ...
   [0.99950033 0.99950033 0.99950033]
   [0.99950033 0.99950033 0.99950033]
   [0.99950033 0.99950033 0.99950033]]

  [[0.99950033 0.99950033 0.99950033]
   [0.99950033 0.99950033 0.99950033]
   [0.99950033 0.99950033 0.99950033]
   ...
   [0.99950033 0.99950033 0.99950033]
   [0.99950033 0.99950033 0.99950033]
   [0.99950033 0.99950033 0.99950033]]

  [[0.99950033 0.99950033 0.99950033]
   [0.99950033 0.99950033 0.99950033]
   [0.99950033 0.99950033 0.99950033]
   ...
   [0.99950033 0.99950033 0.99950033]
   [0.99950033 0.99950033 0.99950033]
   [0.99950033 0.99950033 0.99950033]]

  ...

  [[0.99950033 0.99950033 0.99950033]
   [0.99950033 0.99950033 0.99950033]
   [0.99950033 0.99950033 0.99950033]
   ...
   [0.99950033 0.99950033 0.99950033]
   [0.99950033 0.99950033 0.99950033]
   [0.99950033 0.99950033 0.99950033]]

  [[0.99950033 0.99950033 0.9

In [307]:
K.eval(bn_weights[:, None, None, :]).shape

(10, 1, 1, 3)

In [320]:
out_bn = out_bn * bn_weights[:, None, None, :] + bias[None, None, None, :]

In [314]:
K.int_shape(bias[None, None, None, :])

(1, 1, 1, 3)

In [319]:
bias = K.ones(shape=(3, ))

In [2]:
path_partition = "Data/partition_chr.json"

In [3]:
!python training.py -p {path_partition} --downsample --no_memory --architecture "EfficientNetB2" --chromosom "chr1" --mixed -opt "RAdam" --lookahead -b 64 -e 1

Using TensorFlow backend.
/opt/python3.5/lib/python3.5/site-packages/keras_applications/imagenet_utils.py:279: UserWarning: This model usually expects 1 or 3 input channels. However, it was passed an input_shape with 5 input channels.
  str(input_shape[-1]) + ' input channels.')
Traceback (most recent call last):
  File "training.py", line 194, in <module>
    train(args)
  File "training.py", line 91, in train
    model = create_network((299, 299, 5), getattr(module, args.architecture), mixed=args.mixed) # calls EfficientNetBX from a string 
  File "training.py", line 39, in create_network
    base_model = keras_model(input_tensor=inp, weights=None, include_top=False, input_shape=input_shape, pooling="avg", mixed=mixed)
  File "/home/users/cec1/3DSomaticNet/keras_efficientmixnets/efficientmixnet.py", line 649, in EfficientNetB2
    n_mixture=n_mixture)
  File "/home/users/cec1/3DSomaticNet/keras_efficientmixnets/efficientmixnet.py", line 513, in EfficientNet
    model = Model(inputs, 

In [1]:
from keras_efficientmixnets import efficientmixnet

Using TensorFlow backend.


In [2]:
from keras.layers import *

In [3]:
inp = Input(shape=(299, 299, 5))
   

In [4]:
B2 = efficientmixnet.EfficientNetB2(weights=None, input_tensor=inp, pooling='avg', include_top=False, mixed=False, typeBN=None, n_mixture=None)

In [6]:
def create_network(input_shape, keras_model, mixed=True, typeBN=None):
    '''Conv Network architecture, based on a keras model such as EfficientNets
    '''
    inp = Input(shape=input_shape)
    base_model = keras_model(input_tensor=inp, weights=None, include_top=False, input_shape=input_shape, pooling="avg", mixed=mixed)
    x = base_model.output
    x = Dense(1024, use_bias=True, kernel_initializer=EfficientNetDenseInitializer())(x)
    x = Swish()(x)
    x = Dropout(0.5)(x)
    x = Dense(200, use_bias=True, kernel_initializer=EfficientNetDenseInitializer())(x)
    x = Swish()(x)
    x = Dropout(0.2)(x)
    predictions = Dense(2, activation="softmax", kernel_initializer=EfficientNetDenseInitializer())(x)
    return Model(inp, predictions)

In [3]:
# Input: ["eat", "tea", "tan", "ate", "nat", "bat"],
# Output:
# [
#   ["ate","eat","tea"],
#   ["nat","tan"],
#   ["bat"]
# ]

aet
aet
ant
aet
ant
abt


In [7]:
dic = defaultdict(list)

In [9]:
for x in inp:
    key = "".join(sorted(x))
    dic[key].append(x)

In [14]:
ans = []
for k in dic.values():
    ans.append(k)

In [15]:
ans

[['tan', 'nat'], ['eat', 'tea', 'ate'], ['bat']]